In [1]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import random
import re
import requests
import time

os.chdir("../")

In [33]:
data = open("data/Belgium/2019_general_urls.txt", "r")
links = data.read().split("\n")
links = list(filter(lambda x: x != "", links))
links

['https://resultatselection.belgium.be/fr/election-results/229941',
 'https://resultatselection.belgium.be/fr/election-results/229779',
 'https://resultatselection.belgium.be/fr/election-results/229656',
 'https://resultatselection.belgium.be/fr/election-results/229778',
 'https://resultatselection.belgium.be/fr/election-results/229981',
 'https://resultatselection.belgium.be/fr/election-results/230142',
 'https://resultatselection.belgium.be/fr/election-results/229900',
 'https://resultatselection.belgium.be/fr/election-results/229895',
 'https://resultatselection.belgium.be/fr/election-results/230029',
 'https://resultatselection.belgium.be/fr/election-results/230070',
 'https://resultatselection.belgium.be/fr/election-results/230214',
 'https://resultatselection.belgium.be/fr/election-results/229725',
 'https://resultatselection.belgium.be/fr/election-results/230002',
 'https://resultatselection.belgium.be/fr/election-results/230199',
 'https://resultatselection.belgium.be/fr/electi

In [39]:
output = []
for link in links:
    m = re.search("results\/\d+", link)
    commune_id = m.group(0).split("/")[1]
    data = requests.get(f"https://api.electionresults.belgium.be/election-level/{commune_id}/compare").json()
    frag = pd.DataFrame(data)

    output.append(frag)

df = pd.concat(output, ignore_index=True)
df.head()

In [41]:
df.shape

(1157, 11)

In [40]:
df.to_csv("data/Belgium/2019_general.csv", index=False)

In [26]:
output = []

for link in links:
    print(link)
    m = re.search("results\/\d+", link)
    commune_id = m.group(0).split("/")[1]
    

    r = requests.get(f"https://resultatselection.belgium.be/fr/election-results/chambre-des-repr%C3%A9sentants/2019/commune/{commune_id}")
    soup = BeautifulSoup(r.text)

    try:
        spoilt = int(soup.select(".flex-fill")[1].text.split()[1])

        output.append({
            "party_id": 99,
            "value": spoilt,
            "commune_id": commune_id
        })
    except:
        output.append({
            "party_id": 99,
            "value": 0,
            "commune_id": commune_id
        })

    for item in soup.select(".row-candidate"):
        party_id = item["data-party-id"]
        data = item.text.split()

        output.append({
            "party_id": int(party_id),
            "value": int(data[-3]),
            "commune_id": commune_id
        })

    # time.sleep(random.randint(1, 10) / 10)

229941
229779
229656
229778
229981
230142
229900
229895
230029
230070
230214
229725
230002
230199
230046
230043
230013
229657
229859
229894
229698
230155
230113
229750
229920
230215
229960
230156
230157
230071
229943
229726
229860
230047
230048
230072
229699
229700
230049
230160
230003
230200
229809
229836
229701
229751
229780
229781
229961
229727
230114
229686
229910
230099
229962
229782
230161
230177
229953
230050
230201
229752
229783
230143
230026
229837
230066
230130
229658
229687
229659
229784
229688
229660
230178
230121
229983
229785
229810
229811
229996
230100
229945
229661
229662
229871
230131
229963
229838
230020
229728
229911
230073
230030
230096
230074
230014
230230
229973
229971
229972
229828
230051
229812
229964
230004
230189
230202
230031
229993
230052
230024
229974
229813
229863
230231
230101
230053
229839
230179
229877
229926
229861
229940
229901
229912
229886
229702
229927
230115
229786
229753
230139
230203
230075
229846
230232
230102
229984
229744
229689
230168
229999

AttributeError: 'NoneType' object has no attribute 'group'

In [35]:
df = pd.DataFrame(output)
df.to_csv("data/Belgium/2019_general.csv", index=False)